In [75]:
from google import genai
from secret import api_key

In [4]:
from google import genai
from google.genai import types

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words", config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking
    )
)
print(response.text)

AI works by recognizing patterns in data.


In [5]:
import feedparser
import re
import requests
from datetime import datetime, timedelta
from urllib.parse import quote, urlparse, parse_qs
from collections import defaultdict
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
import warnings
import logging
import time

# Suppress warnings
warnings.filterwarnings('ignore')
logging.getLogger('yfinance').setLevel(logging.CRITICAL)

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [6]:
# Sectors and keywords to monitor
sectors = {
    "AI": [
        "artificial intelligence", "AI", "machine learning", "deep learning",
        "generative AI", "AI chips", "AI infrastructure", "neural networks", "automation",
        "large language model", "LLM", "AI-powered", "AI software", "cognitive computing"
    ],
    "Defence": [
        "defense", "defence", "military", "aerospace", "weapons", "missile systems",
        "defense contractor", "security technology", "cyber defense", "naval systems",
        "defense budget", "aerospace engineering", "army", "defense stocks"
    ],
        "Quantum": [
        "quantum computing", "quantum technology", "quantum mechanics", "quantum encryption",
        "quantum processor", "quantum algorithm", "quantum breakthrough", "quantum chip",
        "quantum sensors", "quantum startup", "quantum research", "quantum cloud",
        "quantum AI", "quantum communication", "superconducting qubits"
    ],

    "Space": [
        "space exploration", "satellite technology", "space stocks", "aerospace startup",
        "space launch", "rocketry", "satellite constellation", "space infrastructure",
        "space economy", "orbital systems", "space tourism", "NASA contracts",
        "SpaceX", "Blue Origin", "space innovation", "low Earth orbit", "satellite imaging"
    ]
    # "Renewable Energy": [
    #     "solar", "solar panels", "wind energy", "wind turbines", "renewable",
    #     "clean energy", "green hydrogen", "hydroelectric", "geothermal",
    #     "energy transition", "carbon neutral", "climate tech", "net zero", "ESG"
    # ],
    # "EV": [
    #     "electric vehicle", "EV", "EVs", "battery technology", "charging infrastructure",
    #     "EV charging stations", "solid-state battery", "autonomous vehicles",
    #     "electric mobility", "EV manufacturer", "EV adoption", "EV stocks"
    # ],
    # "Semiconductor": [
    #     "semiconductor", "chip maker", "foundry", "integrated circuits", "GPU", "CPU",
    #     "chip design", "fabless", "TSMC", "NVIDIA", "chip manufacturing", "AI chips",
    #     "semiconductor supply chain", "wafer", "microchip", "semiconductor stocks"
    # ],
    # "Healthcare": [
    #     "biotech", "biotechnology", "pharmaceutical", "clinical trial", "FDA approval",
    #     "drug development", "healthcare technology", "medical devices", "genomics",
    #     "biopharma", "telehealth", "medtech", "healthcare stocks", "gene therapy",
    #     "diagnostics", "vaccines"
    # ],
    # "Banking": [
    #     "fintech", "digital banking", "financial technology", "neobanks", "lending platforms",
    #     "payment processing", "blockchain", "open banking", "core banking", "interest rates",
    #     "credit risk", "monetary policy", "asset management", "financial services", "banking stocks"
    # ]
}


# News sources to monitor
sources = [
    "fool.com",              # The Motley Fool – excellent for stock recommendations
    "investorplace.com",     # Retail-focused “stocks to buy” articles
    "247wallst.com",         # Consistent stock lists and analyst insights
    "seekingalpha.com",      # Deep research, long/short analysis
    "benzinga.com",          # Frequent analyst rating & target updates
    "zacks.com",             # Quant-based stock ratings & filters
    "tipranks.com",          # Analyst consensus & price targets
    "barrons.com",           # Premium stock and sector insights
    "marketbeat.com",        # Dividend, analyst, and watchlist updates
    "simplywall.st",         # Valuation and recommendation content
    "thestreet.com",         # Technical and stock rating updates
    "investopedia.com",      # Broader financial explainers + “top stock” roundups
    "finance.yahoo.com",     # Aggregates news & analyst calls
    "businessinsider.com",   # Stock lists and market commentary
    "nasdaq.com",            # Company insights and ETF sector writeups
    "markets.businessinsider.com",  # Stock insights & earnings
    "morningstar.com",       # Professional-grade fundamental analysis
    "bloomberg.com",         # Institutional-grade coverage (useful if filtered correctly)
    "cnbc.com",              # Analyst picks & investor segments
    "reuters.com"            # Factual corporate/market reports, good for credibility
]


# Price categories for recommendations
price_categories = {
    "Micro ($1-$5)": (1, 5),
    "Low ($5-$10)": (5, 10),
    "Medium ($10-$15)": (10, 15),
    "Upper ($15-$20)": (15, 20)
}

recommendation_terms = [
    "best stocks to buy",
    "top stocks to buy",
    "stocks to watch",
    "stocks under $10 to buy",
    "undervalued stocks",
    "analyst upgrades",
    "stock picks",
    "buy rating",
    "cheap stocks to buy now",
    "stocks with upside"
]

price_terms = [
    "under $20", "under $15", "under $10", "under $5",
    "cheap stock", "low priced", "affordable stock", "penny stock"
]

# How far back to look for articles (days)
lookback_days = 14

# Minimum unique articles mentioning a ticker to be considered
min_unique_articles = 1

# Enable article content scraping for better ticker extraction
scrape_article_content = True
max_articles_to_scrape = 100  # Limit to avoid too many requests

print(f"✅ Configuration loaded:")
print(f"   • Monitoring {len(sectors)} sectors")
print(f"   • Scanning {len(sources)} news sources")
print(f"   • Price categories: {len(price_categories)}")
print(f"   • Looking back: {lookback_days} days")
print(f"   • Article scraping: {'Enabled' if scrape_article_content else 'Disabled'}")

✅ Configuration loaded:
   • Monitoring 4 sectors
   • Scanning 20 news sources
   • Price categories: 4
   • Looking back: 14 days
   • Article scraping: Enabled


In [7]:
def generate_all_query_combinations_complete(sectors, recommendation_terms, price_terms):
    """
    Generate ALL combinations of [RECOMMENDATION] [SECTOR KEYWORD] [PRICE]
    Using EVERY sector keyword, not just main ones
    """
    all_queries = []

    for sector_name, sector_keywords in sectors.items():
        for rec_term in recommendation_terms:
            for price_term in price_terms:
                for sector_kw in sector_keywords:  # ALL keywords!
                    query = f"{rec_term} {sector_kw} {price_term}"
                    all_queries.append({
                        'query': query,
                        'sector': sector_name,
                        'rec_term': rec_term,
                        'price_term': price_term,
                        'sector_keyword': sector_kw
                    })

    return all_queries


# Generate EVERYTHING
all_queries = generate_all_query_combinations_complete(
    sectors=sectors,
    recommendation_terms=recommendation_terms,
    price_terms=price_terms
)

print("="*80)
print(f"📊 TOTAL QUERY COMBINATIONS: {len(all_queries)}")
print("="*80 + "\n")

# Breakdown by sector
print("Queries per sector:")
for sector_name in sectors.keys():
    sector_queries = [q for q in all_queries if q['sector'] == sector_name]
    num_keywords = len(sectors[sector_name])
    expected = len(recommendation_terms) * len(price_terms) * num_keywords
    print(f"  {sector_name:20} {len(sector_queries):4} queries ({num_keywords} keywords)")

print(f"\n{'='*80}")
print("COMPLETE QUERY LIST:")
print("="*80 + "\n")

# Print EVERY SINGLE QUERY
for i, q in enumerate(all_queries, 1):
    print(f"{i:4}. [{q['sector']:20}] {q['query']}")

print(f"\n{'='*80}")
print(f"✅ Generated {len(all_queries)} total unique query combinations")
print("="*80)

📊 TOTAL QUERY COMBINATIONS: 4800

Queries per sector:
  AI                   1120 queries (14 keywords)
  Defence              1120 queries (14 keywords)
  Quantum              1200 queries (15 keywords)
  Space                1360 queries (17 keywords)

COMPLETE QUERY LIST:

   1. [AI                  ] best stocks to buy artificial intelligence under $20
   2. [AI                  ] best stocks to buy AI under $20
   3. [AI                  ] best stocks to buy machine learning under $20
   4. [AI                  ] best stocks to buy deep learning under $20
   5. [AI                  ] best stocks to buy generative AI under $20
   6. [AI                  ] best stocks to buy AI chips under $20
   7. [AI                  ] best stocks to buy AI infrastructure under $20
   8. [AI                  ] best stocks to buy neural networks under $20
   9. [AI                  ] best stocks to buy automation under $20
  10. [AI                  ] best stocks to buy large language model under 

In [8]:
#WORKS

import concurrent.futures
from threading import Lock

# Progress tracking
progress_lock = Lock()
progress_counter = {'current': 0, 'total': 0}

def fetch_top_20_articles(query_dict, max_articles=20):
    """
    Simple fetch - just get top 20 articles for a query
    NO filtering, NO helper functions
    """
    query = query_dict['query']
    query_encoded = quote(query)
    url = f"https://news.google.com/rss/search?q={query_encoded}&hl=en-US&gl=US&ceid=US:en"

    search_timestamp = datetime.now()

    try:
        feed = feedparser.parse(url)
        articles = []

        for entry in feed.entries[:max_articles]:  # Top 20
            # Parse date
            published_dt = None
            if hasattr(entry, "published_parsed") and entry.published_parsed:
                try:
                    published_dt = datetime(*entry.published_parsed[:6])
                except:
                    pass

            # Calculate age
            days_old = None
            if published_dt:
                days_old = (search_timestamp - published_dt).days

            # Simple article dict
            article = {
                "title": entry.title.strip(),
                "link": entry.link,
                "published": published_dt,
                "published_str": published_dt.strftime('%Y-%m-%d %H:%M') if published_dt else "Unknown",
                "days_old": days_old,
                "search_timestamp": search_timestamp,
                "search_date": search_timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                "query": query,
                "sector": query_dict['sector'],
                "rec_term": query_dict['rec_term'],
                "price_term": query_dict['price_term'],
                "sector_keyword": query_dict['sector_keyword'],
            }

            articles.append(article)

        # Update progress
        with progress_lock:
            progress_counter['current'] += 1
            if progress_counter['current'] % 100 == 0:
                current = progress_counter['current']
                total = progress_counter['total']
                print(f"Progress: {current}/{total} ({current/total*100:.1f}%)")

        return articles

    except Exception as e:
        return []


def fetch_all_articles_simple(all_queries, max_workers=50):
    """
    Fetch top 20 articles for ALL queries - simple & fast
    Returns ALL articles with complete metadata
    """
    print("="*80)
    print(f"🚀 FETCHING: {len(all_queries)} queries with {max_workers} workers")
    print("="*80 + "\n")

    progress_counter['current'] = 0
    progress_counter['total'] = len(all_queries)
    progress_counter['start_time'] = datetime.now()

    all_articles = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(fetch_top_20_articles, query_dict, 20): query_dict
            for query_dict in all_queries
        }

        for future in concurrent.futures.as_completed(futures):
            try:
                articles = future.result()
                all_articles.extend(articles)
            except:
                pass

    elapsed = (datetime.now() - progress_counter['start_time']).total_seconds()

    print(f"\n{'='*80}")
    print(f"✅ COMPLETE")
    print("="*80 + "\n")
    print(f"Time: {elapsed:.1f}s ({elapsed/60:.1f} min)")
    print(f"Rate: {len(all_queries)/elapsed:.1f} queries/sec")
    print(f"Total articles: {len(all_articles)}")

    return all_articles


# RUN IT
raw_articles = fetch_all_articles_simple(all_queries[:20], max_workers=50)

# Show summary
print("\n📊 BY SECTOR:")
sector_counts = {}
for a in raw_articles:
    sector_counts[a['sector']] = sector_counts.get(a['sector'], 0) + 1

for sector, count in sorted(sector_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {sector:20} {count:5} articles")

print(f"\n📰 Sample (first 5):")
for i, a in enumerate(raw_articles[:5], 1):
    print(f"{i}. {a['title'][:70]}...")
    print(f"   Query: {a['query'][:60]}...")
    print(f"   {a['days_old']}d old\n")

🚀 FETCHING: 20 queries with 50 workers


✅ COMPLETE

Time: 1.1s (0.0 min)
Rate: 18.9 queries/sec
Total articles: 400

📊 BY SECTOR:
  AI                     400 articles

📰 Sample (first 5):
1. Top 45 Machine Learning Interview Questions for 2026 - Simplilearn.com...
   Query: best stocks to buy machine learning under $20...
   2d old

2. 50 NEW Artificial Intelligence Statistics (July 2025) - Exploding Topi...
   Query: best stocks to buy machine learning under $20...
   9d old

3. AI in Health Care: 8 Stocks to Buy Now | Investing - Money US News.com...
   Query: best stocks to buy machine learning under $20...
   27d old

4. Multifactor prediction model for stock market analysis based on deep l...
   Query: best stocks to buy machine learning under $20...
   243d old

5. 8 Best Quantum Computing Stocks to Buy in 2025 - The Motley Fool...
   Query: best stocks to buy machine learning under $20...
   53d old



In [10]:
raw_articles

[{'title': 'Top 45 Machine Learning Interview Questions for 2026 - Simplilearn.com',
  'link': 'https://news.google.com/rss/articles/CBMiogFBVV95cUxOOUtfb0Fwc01vZDAtRU9GNkZKQVNBX0tPQW5iV1RURkZMQ1p2cFY4WWhPQ1hTRE5oUUNFUWR2YXlROUE0UUk1LUdHRVo5Y1YzSlgzM0EzUUY5YUNJVFdHYU13WjU0ZnFLX3NFQWEtNWhWVTYxRmNBWVd1amRtdmota2ZpNG4xaW1mM1JBXzh5SzB4LWktOTAyejNiU0tWNHI4N2c?oc=5',
  'published': datetime.datetime(2025, 10, 10, 7, 0),
  'published_str': '2025-10-10 07:00',
  'days_old': 2,
  'search_timestamp': datetime.datetime(2025, 10, 13, 1, 37, 10, 959964),
  'search_date': '2025-10-13 01:37:10',
  'query': 'best stocks to buy machine learning under $20',
  'sector': 'AI',
  'rec_term': 'best stocks to buy',
  'price_term': 'under $20',
  'sector_keyword': 'machine learning'},
 {'title': '50 NEW Artificial Intelligence Statistics (July 2025) - Exploding Topics',
  'link': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTFBRR1RxSDRTNlhSREl4Ty12bGN5UmZEVEZtZHB5OXprS1p2S0VtSFlyVmx2ZGY4OS11ZDhjODVGMTcwd

In [13]:
def get_stock_recommendations_improved(articles):
    prompt = """You are a financial analyst. Classify if each article is a STOCK RECOMMENDATION (tells readers which stocks to buy/sell/watch).

STOCK RECOMMENDATIONS (return true):
- "Top 5 AI Stocks to Buy Now"
- "3 Penny Stocks Under $10 to Watch"
- "Best Biotech Stocks for 2025"

NOT RECOMMENDATIONS (return false):
- News: "Tesla Stock Rises 5%"
- Earnings: "Apple Beats Q3 Estimates"
- Education: "How to Invest in AI"

Return JSON array [true,false,...] for these titles:
""" + "\n".join([f"{i+1}. {a['title']}" for i, a in enumerate(articles)])

    response = client.models.generate_content(
        model="gemini-2.0-flash-thinking-exp",  # Thinking-optimized model
        contents=prompt,
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(thinking_budget=10000),  # Enable thinking!
            response_mime_type="application/json",
            temperature=0.1
        )
    )
    results = json.loads(re.sub(r'\s+', '', response.text))
    return [a for a, is_rec in zip(articles, results) if is_rec]

# Test
recs = get_stock_recommendations_improved(raw_articles[:20])
print(f"✅ {len(recs)} recommendations:")
for r in recs:
    print(f"  • {r['title']}")

✅ 5 recommendations:
  • AI in Health Care: 8 Stocks to Buy Now | Investing - Money US News.com
  • 8 Best Quantum Computing Stocks to Buy in 2025 - The Motley Fool
  • On Fire: 5 Best Artificial Intelligence Penny Stocks - MarketBeat
  • Best Artificial Intelligence Stocks in India 2025 - Groww
  • 7 Best Machine Learning Stocks to Buy in 2025 - The Motley Fool


In [20]:
import asyncio
from playwright.async_api import async_playwright
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

async def resolve_article_url(page, article, retry=0):
    """Resolve Google News URL to actual article URL with multiple strategies"""
    try:
        # Strategy 1: Direct navigation
        response = await page.goto(
            article['link'],
            wait_until='load',
            timeout=45000
        )

        if not response or response.status >= 400:
            logger.warning(f"Bad response: {response.status if response else 'None'}")
            if retry < 2:
                await asyncio.sleep(2)
                return await resolve_article_url(page, article, retry + 1)
            return None

        # Handle consent BEFORE waiting for redirect
        await handle_consent(page)

        # Strategy 2: Check for Google News interstitial and click through
        try:
            # Look for "Go to article" or similar links
            article_links = await page.locator('a[href]').all()
            for link in article_links[:20]:  # Check first 20 links
                text = await link.inner_text()
                text_lower = text.lower()
                if any(phrase in text_lower for phrase in ['read full', 'go to', 'view article', 'continue to']):
                    logger.info(f"Found interstitial link: {text[:30]}")
                    await link.click()
                    await page.wait_for_load_state('domcontentloaded', timeout=10000)
                    break
        except Exception as e:
            logger.debug(f"Interstitial check: {e}")

        # Wait for redirect with multiple strategies
        redirected = await wait_for_redirect(page)

        if not redirected:
            # Strategy 3: If no redirect, try finding the first external link
            try:
                external_link = await page.locator('a[href^="http"]:not([href*="google.com"])').first
                if await external_link.is_visible(timeout=2000):
                    href = await external_link.get_attribute('href')
                    if href and 'google.com' not in href:
                        logger.info(f"Found external link: {href[:50]}")
                        await external_link.click()
                        await page.wait_for_load_state('domcontentloaded', timeout=10000)
            except Exception as e:
                logger.debug(f"External link search: {e}")

        await asyncio.sleep(1.5)
        final_url = page.url

        # Validate URL
        if not is_valid_url(final_url):
            if retry < 2:
                logger.info(f"Invalid URL, retry {retry + 1}: {article['title'][:40]}")
                await asyncio.sleep(2)
                return await resolve_article_url(page, article, retry + 1)
            return None

        return final_url

    except asyncio.TimeoutError:
        logger.warning(f"Timeout: {article['title'][:40]}")
        if retry < 2:
            await asyncio.sleep(2)
            return await resolve_article_url(page, article, retry + 1)
        return None
    except Exception as e:
        logger.error(f"Error: {article['title'][:40]} - {type(e).__name__}: {e}")
        if retry < 1:
            return await resolve_article_url(page, article, retry + 1)
        return None


async def handle_consent(page):
    """Handle consent dialogs with multiple strategies"""
    await asyncio.sleep(0.8)

    try:
        # Strategy 1: Text-based button search
        consent_texts = ['reject', 'decline', 'no thanks', 'dismiss', 'continue without', 'not now']
        buttons = await page.locator("button").all()

        for btn in buttons:
            try:
                text = (await btn.inner_text()).lower()
                if any(keyword in text for keyword in consent_texts):
                    if await btn.is_visible():
                        await btn.click()
                        await asyncio.sleep(1)
                        logger.debug("Clicked consent button")
                        return
            except:
                continue

        # Strategy 2: Selector-based search
        consent_selectors = [
            "button:has-text('Reject')",
            "button:has-text('Decline')",
            "[aria-label*='reject' i]",
            "[class*='reject' i]",
            "[class*='decline' i]",
            ".fc-cta-do-not-consent",  # Common consent framework
            "#onetrust-reject-all-handler"  # OneTrust
        ]

        for selector in consent_selectors:
            try:
                btn = page.locator(selector).first
                if await btn.is_visible(timeout=1000):
                    await btn.click()
                    await asyncio.sleep(1)
                    logger.debug(f"Clicked consent: {selector}")
                    return
            except:
                continue

    except Exception as e:
        logger.debug(f"Consent handling: {e}")


async def wait_for_redirect(page, timeout=12000):
    """Wait for redirect away from Google with multiple strategies"""
    try:
        # Strategy 1: Wait for URL change
        await page.wait_for_function(
            "window.location.href.indexOf('google.com') === -1 && window.location.href.indexOf('chrome-error') === -1",
            timeout=timeout
        )
        return True
    except:
        pass

    try:
        # Strategy 2: Wait for network idle
        await page.wait_for_load_state('networkidle', timeout=8000)
        if 'google.com' not in page.url:
            return True
    except:
        pass

    try:
        # Strategy 3: Wait for article indicators (title, content)
        await page.wait_for_selector('article, [class*="article"], [id*="article"], h1', timeout=5000)
        if 'google.com' not in page.url:
            return True
    except:
        pass

    return False


def is_valid_url(url):
    """Check if URL is valid and not an error"""
    if not url:
        return False

    invalid_patterns = ['google.com', 'chrome-error', 'about:blank', 'data:', 'javascript:']

    return not any(pattern in url.lower() for pattern in invalid_patterns)


async def resolve_all_urls(articles, max_concurrent=3):
    """Resolve URLs with optimized browser settings"""
    print(f"🔗 Resolving URLs for {len(articles)} articles...\n")

    async with async_playwright() as p:
        # Launch with stealth settings
        browser = await p.chromium.launch(
            headless=True,
            args=[
                '--disable-blink-features=AutomationControlled',
                '--disable-dev-shm-usage',
                '--no-sandbox',
                '--disable-setuid-sandbox',
                '--disable-web-security'
            ]
        )

        # Realistic browser context
        context = await browser.new_context(
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            viewport={'width': 1920, 'height': 1080},
            java_script_enabled=True,
            bypass_csp=True,
            ignore_https_errors=True
        )

        # Add stealth script
        await context.add_init_script("""
            Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined
            });
        """)

        semaphore = asyncio.Semaphore(max_concurrent)

        async def process_article(article):
            async with semaphore:
                page = await context.new_page()
                try:
                    url = await resolve_article_url(page, article)
                    return article, url
                finally:
                    await page.close()

        tasks = [process_article(article) for article in articles]
        results = await asyncio.gather(*tasks, return_exceptions=True)

        await context.close()
        await browser.close()

    # Process results
    success = 0
    for result in results:
        if isinstance(result, Exception):
            logger.error(f"Task failed: {result}")
            continue

        article, url = result
        article['resolved_url'] = url

        if url:
            success += 1
            print(f"✅ {article['title'][:60]}...")
            print(f"   → {url[:80]}\n")
        else:
            print(f"❌ {article['title'][:60]}...")
            print(f"   → Failed to resolve\n")

    success_rate = (success/len(articles)*100) if articles else 0
    print(f"\n{'='*80}")
    print(f"✅ Resolved: {success}/{len(articles)} ({success_rate:.1f}%)")
    print(f"{'='*80}")

    return articles


# Run it
recs_with_urls = await resolve_all_urls(recs, max_concurrent=3)

# Show valid results only
resolved = [r for r in recs_with_urls if r.get('resolved_url') and is_valid_url(r['resolved_url'])]
print(f"\n🎯 {len(resolved)} articles ready to scrape:")
for r in resolved:
    print(f"  {r['title'][:50]}...")
    print(f"    {r['resolved_url']}\n")

🔗 Resolving URLs for 5 articles...



INFO:__main__:Invalid URL, retry 1: AI in Health Care: 8 Stocks to Buy Now |
INFO:__main__:Invalid URL, retry 2: AI in Health Care: 8 Stocks to Buy Now |


❌ AI in Health Care: 8 Stocks to Buy Now | Investing - Money U...
   → Failed to resolve

✅ 8 Best Quantum Computing Stocks to Buy in 2025 - The Motley ...
   → https://www.fool.com/investing/stock-market/market-sectors/information-technolog

✅ On Fire: 5 Best Artificial Intelligence Penny Stocks - Marke...
   → https://www.marketbeat.com/stock-ideas/5-best-artificial-intelligence-penny-stoc

✅ Best Artificial Intelligence Stocks in India 2025 - Groww...
   → https://groww.in/blog/best-artificial-intelligence-stocks-in-india

✅ 7 Best Machine Learning Stocks to Buy in 2025 - The Motley F...
   → https://www.fool.com/investing/stock-market/market-sectors/information-technolog


✅ Resolved: 4/5 (80.0%)

🎯 4 articles ready to scrape:
  8 Best Quantum Computing Stocks to Buy in 2025 - T...
    https://www.fool.com/investing/stock-market/market-sectors/information-technology/ai-stocks/quantum-computing-stocks/

  On Fire: 5 Best Artificial Intelligence Penny Stoc...
    https://www.marketbe

In [26]:
async def scrape_page_text(url):
    """Simple: Open page and get all text content"""
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            await page.goto(url, timeout=30000)
            await page.wait_for_timeout(2000)

            text = await page.evaluate('() => document.body.innerText')

            await browser.close()
            return text

        except Exception as e:
            print(f"Error: {e}")
            await browser.close()
            return None


async def add_text_to_recs(recs, max_concurrent=3):
    """Scrape text for all recs and add as attribute"""
    print(f"📚 Scraping text for {len(recs)} articles...\n")

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()

        semaphore = asyncio.Semaphore(max_concurrent)

        async def scrape_rec(rec):
            async with semaphore:
                # Skip if no URL
                if not rec.get('resolved_url'):
                    rec['text'] = None
                    print(f"⏭️  Skipped (no URL): {rec['title'][:50]}")
                    return

                page = await context.new_page()
                try:
                    await page.goto(rec['resolved_url'], timeout=30000)
                    await page.wait_for_timeout(2000)

                    text = await page.evaluate('() => document.body.innerText')
                    rec['text'] = text

                    print(f"✅ {rec['title'][:50]}... ({len(text)} chars)")

                except Exception as e:
                    rec['text'] = None
                    print(f"❌ {rec['title'][:50]}... - {e}")
                finally:
                    await page.close()

        # Scrape all recs
        tasks = [scrape_rec(rec) for rec in recs]
        await asyncio.gather(*tasks)

        await context.close()
        await browser.close()

    # Summary
    success = sum(1 for r in recs if r.get('text'))
    print(f"\n{'='*80}")
    print(f"✅ Scraped: {success}/{len(recs)} articles")
    print(f"{'='*80}")

    return recs


# Add text to your recs
recs = await add_text_to_recs(recs_with_urls, max_concurrent=3)

# Check results
for rec in recs:
    if rec.get('text'):
        print(f"\n{rec['title']}")
        print(f"URL: {rec['resolved_url']}")
        print(f"Text length: {len(rec['text'])} characters")
        print(f"Preview: {rec['text'][:200]}...\n")

📚 Scraping text for 5 articles...

⏭️  Skipped (no URL): AI in Health Care: 8 Stocks to Buy Now | Investing
✅ Best Artificial Intelligence Stocks in India 2025 ... (16506 chars)
✅ 8 Best Quantum Computing Stocks to Buy in 2025 - T... (20159 chars)
✅ On Fire: 5 Best Artificial Intelligence Penny Stoc... (16954 chars)
✅ 7 Best Machine Learning Stocks to Buy in 2025 - Th... (21097 chars)

✅ Scraped: 4/5 articles

8 Best Quantum Computing Stocks to Buy in 2025 - The Motley Fool
URL: https://www.fool.com/investing/stock-market/market-sectors/information-technology/ai-stocks/quantum-computing-stocks/
Text length: 20159 characters
Preview: Skip to main content
Enable accessibility for low vision
Open the accessibility menu
▲ S&P 500 +187% | ▲ Stock Advisor +1061% JOIN THE MOTLEY FOOL
ACCESSIBILITY
LOG IN
HELP
Our Services
Stock Market N...


On Fire: 5 Best Artificial Intelligence Penny Stocks - MarketBeat
URL: https://www.marketbeat.com/stock-ideas/5-best-artificial-intelligence-penny-stocks

In [31]:
text = recs[2]['text']

'Skip to main content\nCould Target’s Week of Discounts Come Full Circle for Investors?Trump\'s Law S.1582: $21T Dollar Revolution Coming (Ad)3 Reasons to Buy Sprouts Farmers Market Ahead of EarningsMicrosoft 365 Premium Marks the Next Phase of AI MonetizationStunning new initiative unfolding in the White House? (Ad)California oil workers face an uncertain future in the state\'s energy transitionTrump says inflation is \'defeated\' and the Fed has cut rates, yet prices remain too high for manyClaim Your Share of $5.39 BILLION in AI Equity Checks (Ad)China vows to stand firm against Trump\'s tariff threat. He urges Beijing to be less confrontationalVance warns \'deeper\' cuts ahead for federal workers as shutdown enters 12th dayCould Target’s Week of Discounts Come Full Circle for Investors?Trump\'s Law S.1582: $21T Dollar Revolution Coming (Ad)3 Reasons to Buy Sprouts Farmers Market Ahead of EarningsMicrosoft 365 Premium Marks the Next Phase of AI MonetizationStunning new initiative un

In [58]:
import json
import re
from google import genai
from google.genai import types

def enrich_recs_with_tickers_and_notes(recs, api_key):
    """
    For each dict in recs, call Gemini to extract stock tickers AND investment rationale.
    Adds 'tickers_list_gemini' (list of tickers) and 'ticker_notes' (dict mapping ticker to notes).
    """
    client = genai.Client(api_key=api_key)

    for idx, rec in enumerate(recs):
        text = rec.get("text", "")
        if not text:
            rec["tickers_list_gemini"] = []
            rec["ticker_notes"] = {}
            continue

        prompt = f"""Extract all stock tickers mentioned in the following article and provide investment rationale for each.

Return ONLY a JSON object with this structure:
{{
  "stocks": [
    {{
      "ticker": "AAPL",
      "notes": "Brief investment rationale or key points about why this stock is mentioned (2-3 sentences max)"
    }}
  ]
}}

Rules:
- Only include valid stock tickers (uppercase, 1-5 characters)
- Exclude index tickers like S&P, DJI, NASDAQ
- Extract the key investment thesis or rationale for each stock
- Keep notes concise (under 200 characters if possible)
- If no specific rationale is given, summarize what the article says about the company

Article Text:
\"\"\"{text[:8000]}\"\"\"

Output only valid JSON, no other text.
"""

        try:
            print(f"Processing rec {idx + 1}/{len(recs)}: {rec.get('title', '')[:60]}...")

            response = client.models.generate_content(
                model="gemini-2.0-flash-exp",
                contents=prompt,
                config=types.GenerateContentConfig(
                    temperature=0.3,  # Lower temperature for more consistent JSON
                    thinking_config=types.ThinkingConfig(thinking_budget=0)
                )
            )
            response_text = response.text.strip()

            # Extract JSON object
            match = re.search(r'\{.*\}', response_text, re.DOTALL)
            if match:
                json_text = match.group(0)
                stocks_data = json.loads(json_text)
                stocks_list = stocks_data.get("stocks", [])

                # Build tickers list and notes dict
                tickers_list = []
                ticker_notes = {}

                for stock in stocks_list:
                    ticker = stock.get("ticker", "").upper()
                    notes = stock.get("notes", "Mentioned in article.")

                    if ticker and len(ticker) <= 5:  # Valid ticker length
                        tickers_list.append(ticker)
                        ticker_notes[ticker] = notes

                rec["tickers_list_gemini"] = tickers_list
                rec["ticker_notes"] = ticker_notes

                print(f"  ✅ Found {len(tickers_list)} tickers: {', '.join(tickers_list[:5])}...")
            else:
                print(f"  ⚠️  No valid JSON found")
                rec["tickers_list_gemini"] = []
                rec["ticker_notes"] = {}

        except Exception as e:
            print(f"  ❌ Error: {e}")
            rec["tickers_list_gemini"] = []
            rec["ticker_notes"] = {}

    return recs


# Usage
recs_enriched = enrich_recs_with_tickers_and_notes(recs_with_urls, api_key)

# Preview results
for rec in recs_enriched[:2]:
    print(f"\n{'='*80}")
    print(f"Article: {rec['title']}")
    print(f"Tickers found: {len(rec.get('tickers_list_gemini', []))}")
    print("\nTicker Notes:")
    for ticker, notes in rec.get('ticker_notes', {}).items():
        print(f"  {ticker}: {notes[:100]}...")

INFO:google_genai.models:AFC is enabled with max remote calls: 10.


Processing rec 2/5: 8 Best Quantum Computing Stocks to Buy in 2025 - The Motley ...


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC is enabled with max remote calls: 10.


  ✅ Found 11 tickers: IONQ, RGTI, ARQQ, QUBT, NVDA...
Processing rec 3/5: On Fire: 5 Best Artificial Intelligence Penny Stocks - Marke...


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC is enabled with max remote calls: 10.


  ✅ Found 7 tickers: BBAI, RR, AISP, PLTR, LHX...
Processing rec 4/5: Best Artificial Intelligence Stocks in India 2025 - Groww...


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC is enabled with max remote calls: 10.


  ✅ Found 2 tickers: AFFLE, BOSCH...
Processing rec 5/5: 7 Best Machine Learning Stocks to Buy in 2025 - The Motley F...


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent "HTTP/1.1 200 OK"


  ✅ Found 11 tickers: IBM, GOOGL, GOOG, NFLX, NVDA...

Article: AI in Health Care: 8 Stocks to Buy Now | Investing - Money US News.com
Tickers found: 0

Ticker Notes:

Article: 8 Best Quantum Computing Stocks to Buy in 2025 - The Motley Fool
Tickers found: 11

Ticker Notes:
  IONQ: A quantum computing pure-play company developing quantum computing hardware and making its systems a...
  RGTI: A small, high-risk business that went public early in the COVID-19 pandemic. Their stock soared in 2...
  ARQQ: A company creating software for quantum computers, specifically security software....
  QUBT: An integrated hardware and software company in the quantum computing space....
  NVDA: A semiconductor designer increasing its activity in the quantum realm....
  MSFT: A software titan researching and developing quantum computing technology, including hardware and sof...
  HON: A large industrial company with a hand in the development of quantum computers and related technolog...
  IBM: A large 

In [72]:
import asyncio
import aiohttp
from collections import defaultdict

async def fetch_stock_price(session, ticker):
    """Fetch current stock price with better error handling"""
    try:
        # Yahoo Finance API v8
        url = f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

        async with session.get(url, headers=headers, timeout=10) as response:
            if response.status == 200:
                data = await response.json()
                result = data.get('chart', {}).get('result', [])
                if result:
                    price = result[0].get('meta', {}).get('regularMarketPrice')
                    if price:
                        print(f"✅ {ticker}: ${price:.2f}")
                        return ticker, price

            print(f"⚠️  {ticker}: Status {response.status}")
            return ticker, None

    except asyncio.TimeoutError:
        print(f"⏱️  {ticker}: Timeout")
        return ticker, None
    except Exception as e:
        print(f"❌ {ticker}: {type(e).__name__}")
        return ticker, None


async def enrich_stocks_with_prices(recs):
    """Aggregate stocks from all recs and fetch prices"""

    # Aggregate stocks by ticker
    stock_sources = defaultdict(list)

    for rec in recs:
        tickers = rec.get('tickers_list_gemini', [])
        ticker_notes = rec.get('ticker_notes', {})

        for ticker in tickers:
            # Filter out obvious non-stocks (more comprehensive)
            excluded = ['S&P', 'DJI', 'NASDAQ', 'ETF', 'AI', 'GPU', 'CEO',
                       'IPO', 'USA', 'UK', 'EU', 'API', 'AWS', 'COVID',
                       'SPAC', 'RCS', 'CSP', 'IT', 'Q1', 'Q2', 'Q3', 'Q4']

            if ticker in excluded or len(ticker) > 5 or len(ticker) < 1:
                continue

            stock_sources[ticker].append({
                'title': rec['title'],
                'url': rec.get('resolved_url'),
                'published': rec.get('published_str'),
                'notes': ticker_notes.get(ticker, 'Mentioned in article.')
            })

    print(f"\n📊 Found {len(stock_sources)} unique tickers across all articles\n")

    # Fetch prices for all tickers
    connector = aiohttp.TCPConnector(limit=10)
    timeout = aiohttp.ClientTimeout(total=30)

    async with aiohttp.ClientSession(connector=connector, timeout=timeout) as session:
        tasks = [fetch_stock_price(session, ticker) for ticker in stock_sources.keys()]
        price_results = await asyncio.gather(*tasks)

    # Build enriched stock data
    stocks = []
    failed_tickers = []

    for ticker, price in price_results:
        if price:  # Only include stocks we successfully priced
            stock_data = {
                'ticker': ticker,
                'price': price,
                'sources': stock_sources[ticker],
                'source_count': len(stock_sources[ticker])
            }
            stocks.append(stock_data)
        else:
            failed_tickers.append(ticker)

    # Sort by number of mentions (most mentioned first)
    stocks.sort(key=lambda x: x['source_count'], reverse=True)

    print(f"\n{'='*80}")
    print(f"✅ Successfully priced: {len(stocks)} stocks")
    print(f"❌ Failed to price: {len(failed_tickers)} tickers")
    if failed_tickers:
        print(f"Failed tickers: {', '.join(failed_tickers[:20])}")
    print(f"{'='*80}\n")

    return stocks


# Run it
enriched_stocks = await enrich_stocks_with_prices(recs_enriched)

# Preview top stocks
print("\n🎯 Top 10 Stocks by Mentions:\n")
for i, stock in enumerate(enriched_stocks[:10], 1):
    print(f"{i}. {stock['ticker']}: ${stock['price']:.2f}")
    print(f"   Mentioned in {stock['source_count']} article(s)")
    print(f"   First source: {stock['sources'][0]['title'][:60]}...")
    print()


📊 Found 28 unique tickers across all articles

✅ HON: $200.91
✅ ARQQ: $48.52
✅ IBM: $277.82
✅ RGTI: $43.92
✅ INTC: $36.33
✅ QUBT: $19.02
✅ NVDA: $183.04
✅ IONQ: $70.65
✅ MSFT: $510.96
✅ QBTS: $33.02
✅ LHX: $292.34
✅ BBAI: $7.22
✅ AMZN: $216.37
✅ PLTR: $175.44
✅ AISP: $5.13
✅ RR: $5.92
✅ SERV: $14.90
⚠️  AFFLE: Status 404
✅ UBER: $93.40
⚠️  BOSCH: Status 404
✅ GOOGL: $236.57
✅ NFLX: $1220.08
✅ GOOG: $237.49
✅ TSLA: $413.49
✅ NOW: $888.71
✅ ACN: $240.94
✅ SNOW: $242.17
✅ CRWD: $493.66

✅ Successfully priced: 26 stocks
❌ Failed to price: 2 tickers
Failed tickers: AFFLE, BOSCH


🎯 Top 10 Stocks by Mentions:

1. NVDA: $183.04
   Mentioned in 2 article(s)
   First source: 8 Best Quantum Computing Stocks to Buy in 2025 - The Motley ...

2. IBM: $277.82
   Mentioned in 2 article(s)
   First source: 8 Best Quantum Computing Stocks to Buy in 2025 - The Motley ...

3. PLTR: $175.44
   Mentioned in 2 article(s)
   First source: On Fire: 5 Best Artificial Intelligence Penny Stocks - Marke...

4. I

In [73]:
def prepare_stocks_for_interactive_ui(enriched_stocks):
    """Format stock data for interactive React UI"""
    formatted_stocks = []

    for stock in enriched_stocks:
        formatted_stocks.append({
            'ticker': stock['ticker'],
            'price': round(stock['price'], 2),
            'source_count': stock['source_count'],
            'sources': stock['sources']  # Already has notes from Gemini!
        })

    return formatted_stocks

# Prepare your data
ui_stocks = prepare_stocks_for_interactive_ui(enriched_stocks)
print(f"✅ Prepared {len(ui_stocks)} stocks for interactive UI")

✅ Prepared 26 stocks for interactive UI


In [74]:
import json
import webbrowser
import os

def generate_interactive_stock_screener(stocks_data, output_file='stock_screener.html'):
    """Generate fully interactive stock screener with animations and features"""

    stocks_json = json.dumps(stocks_data, indent=2)

    html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Stock Investment Screener</title>
    <script crossorigin src="https://unpkg.com/react@18/umd/react.production.min.js"></script>
    <script crossorigin src="https://unpkg.com/react-dom@18/umd/react-dom.production.min.js"></script>
    <script src="https://unpkg.com/@babel/standalone/babel.min.js"></script>
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        @keyframes fadeIn {{
            from {{ opacity: 0; transform: translateY(20px); }}
            to {{ opacity: 1; transform: translateY(0); }}
        }}
        .fade-in {{
            animation: fadeIn 0.5s ease-out;
        }}
        .line-clamp-3 {{
            display: -webkit-box;
            -webkit-line-clamp: 3;
            -webkit-box-orient: vertical;
            overflow: hidden;
        }}
    </style>
</head>
<body>
    <div id="root"></div>

    <script type="text/babel">
        const {{ useState, useMemo, useEffect }} = React;

        // Lucide icons as inline SVG components
        const TrendingUp = ({{ size = 24, className = "" }}) => (
            <svg xmlns="http://www.w3.org/2000/svg" width={{size}} height={{size}} viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round" className={{className}}>
                <polyline points="22 7 13.5 15.5 8.5 10.5 2 17"></polyline>
                <polyline points="16 7 22 7 22 13"></polyline>
            </svg>
        );

        const Filter = ({{ size = 24, className = "" }}) => (
            <svg xmlns="http://www.w3.org/2000/svg" width={{size}} height={{size}} viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round" className={{className}}>
                <polygon points="22 3 2 3 10 12.46 10 19 14 21 14 12.46 22 3"></polygon>
            </svg>
        );

        const DollarSign = ({{ size = 24, className = "" }}) => (
            <svg xmlns="http://www.w3.org/2000/svg" width={{size}} height={{size}} viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round" className={{className}}>
                <line x1="12" y1="1" x2="12" y2="23"></line>
                <path d="M17 5H9.5a3.5 3.5 0 0 0 0 7h5a3.5 3.5 0 0 1 0 7H6"></path>
            </svg>
        );

        const FileText = ({{ size = 24, className = "" }}) => (
            <svg xmlns="http://www.w3.org/2000/svg" width={{size}} height={{size}} viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round" className={{className}}>
                <path d="M14 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V8z"></path>
                <polyline points="14 2 14 8 20 8"></polyline>
                <line x1="16" y1="13" x2="8" y2="13"></line>
                <line x1="16" y1="17" x2="8" y2="17"></line>
                <polyline points="10 9 9 9 8 9"></polyline>
            </svg>
        );

        const ExternalLink = ({{ size = 24, className = "" }}) => (
            <svg xmlns="http://www.w3.org/2000/svg" width={{size}} height={{size}} viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round" className={{className}}>
                <path d="M18 13v6a2 2 0 0 1-2 2H5a2 2 0 0 1-2-2V8a2 2 0 0 1 2-2h6"></path>
                <polyline points="15 3 21 3 21 9"></polyline>
                <line x1="10" y1="14" x2="21" y2="3"></line>
            </svg>
        );

        const X = ({{ size = 24, className = "" }}) => (
            <svg xmlns="http://www.w3.org/2000/svg" width={{size}} height={{size}} viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round" className={{className}}>
                <line x1="18" y1="6" x2="6" y2="18"></line>
                <line x1="6" y1="6" x2="18" y2="18"></line>
            </svg>
        );

        const BarChart = ({{ size = 24, className = "" }}) => (
            <svg xmlns="http://www.w3.org/2000/svg" width={{size}} height={{size}} viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round" className={{className}}>
                <line x1="12" y1="20" x2="12" y2="10"></line>
                <line x1="18" y1="20" x2="18" y2="4"></line>
                <line x1="6" y1="20" x2="6" y2="16"></line>
            </svg>
        );

        // Your stock data
        const stocksData = {stocks_json};

        function StockScreener() {{
          const [priceFilter, setPriceFilter] = useState('all');
          const [sortBy, setSortBy] = useState('mentions');
          const [searchQuery, setSearchQuery] = useState('');
          const [selectedStock, setSelectedStock] = useState(null);
          const [showStats, setShowStats] = useState(false);

          const filteredStocks = useMemo(() => {{
            let filtered = stocksData;

            if (priceFilter !== 'all') {{
              const maxPrice = parseInt(priceFilter);
              filtered = filtered.filter(s => s.price <= maxPrice);
            }}

            if (searchQuery) {{
              filtered = filtered.filter(s =>
                s.ticker.toLowerCase().includes(searchQuery.toLowerCase())
              );
            }}

            if (sortBy === 'price-asc') {{
              filtered = [...filtered].sort((a, b) => a.price - b.price);
            }} else if (sortBy === 'price-desc') {{
              filtered = [...filtered].sort((a, b) => b.price - a.price);
            }} else if (sortBy === 'mentions') {{
              filtered = [...filtered].sort((a, b) => b.source_count - a.source_count);
            }}

            return filtered;
          }}, [priceFilter, sortBy, searchQuery]);

          // Calculate stats
          const stats = useMemo(() => {{
            const prices = filteredStocks.map(s => s.price);
            const avgPrice = prices.reduce((a, b) => a + b, 0) / prices.length || 0;
            const under20 = filteredStocks.filter(s => s.price < 20).length;
            const totalMentions = filteredStocks.reduce((sum, s) => sum + s.source_count, 0);

            return {{
              total: filteredStocks.length,
              avgPrice: avgPrice.toFixed(2),
              under20,
              totalMentions
            }};
          }}, [filteredStocks]);

          return (
            <div className="min-h-screen bg-gradient-to-br from-slate-50 via-blue-50 to-slate-100 p-4 md:p-6">
              <div className="max-w-7xl mx-auto">
                {{/* Header */}}
                <div className="mb-6 md:mb-8 fade-in">
                  <div className="flex items-center justify-between flex-wrap gap-4">
                    <div>
                      <h1 className="text-3xl md:text-4xl font-bold text-slate-900 mb-2 flex items-center gap-3">
                        <TrendingUp size={{36}} className="text-emerald-600" />
                        Stock Investment Screener
                      </h1>
                      <p className="text-slate-600">
                        Curated investment opportunities from {{stats.totalMentions}} article mentions
                      </p>
                    </div>
                    <button
                      onClick={{() => setShowStats(!showStats)}}
                      className="px-4 py-2 bg-white rounded-lg shadow-sm hover:shadow-md transition-all flex items-center gap-2 text-slate-700 font-medium"
                    >
                      <BarChart size={{20}} />
                      Stats
                    </button>
                  </div>
                </div>

                {{/* Stats Panel */}}
                {{showStats && (
                  <div className="mb-6 grid grid-cols-2 md:grid-cols-4 gap-4 fade-in">
                    <div className="bg-white rounded-xl p-4 shadow-sm">
                      <div className="text-2xl font-bold text-emerald-600">{{stats.total}}</div>
                      <div className="text-sm text-slate-600">Total Stocks</div>
                    </div>
                    <div className="bg-white rounded-xl p-4 shadow-sm">
                      <div className="text-2xl font-bold text-blue-600">${{stats.avgPrice}}</div>
                      <div className="text-sm text-slate-600">Avg Price</div>
                    </div>
                    <div className="bg-white rounded-xl p-4 shadow-sm">
                      <div className="text-2xl font-bold text-purple-600">{{stats.under20}}</div>
                      <div className="text-sm text-slate-600">Under $20</div>
                    </div>
                    <div className="bg-white rounded-xl p-4 shadow-sm">
                      <div className="text-2xl font-bold text-orange-600">{{stats.totalMentions}}</div>
                      <div className="text-sm text-slate-600">Total Mentions</div>
                    </div>
                  </div>
                )}}

                {{/* Filters */}}
                <div className="bg-white rounded-xl shadow-md p-4 md:p-6 mb-6 fade-in">
                  <div className="flex items-center gap-2 mb-4">
                    <Filter size={{20}} className="text-slate-600" />
                    <h2 className="text-lg font-semibold text-slate-900">Filters</h2>
                  </div>

                  <div className="grid grid-cols-1 md:grid-cols-3 gap-4">
                    <div>
                      <label className="block text-sm font-medium text-slate-700 mb-2">
                        Search Ticker
                      </label>
                      <input
                        type="text"
                        placeholder="e.g., NVDA, MSFT..."
                        value={{searchQuery}}
                        onChange={{(e) => setSearchQuery(e.target.value)}}
                        className="w-full px-4 py-2 border border-slate-300 rounded-lg focus:ring-2 focus:ring-emerald-500 focus:border-transparent transition-all"
                      />
                    </div>

                    <div>
                      <label className="block text-sm font-medium text-slate-700 mb-2">
                        Maximum Price
                      </label>
                      <select
                        value={{priceFilter}}
                        onChange={{(e) => setPriceFilter(e.target.value)}}
                        className="w-full px-4 py-2 border border-slate-300 rounded-lg focus:ring-2 focus:ring-emerald-500 focus:border-transparent transition-all"
                      >
                        <option value="all">All Prices</option>
                        <option value="5">Under $5</option>
                        <option value="10">Under $10</option>
                        <option value="15">Under $15</option>
                        <option value="20">Under $20</option>
                        <option value="50">Under $50</option>
                        <option value="100">Under $100</option>
                      </select>
                    </div>

                    <div>
                      <label className="block text-sm font-medium text-slate-700 mb-2">
                        Sort By
                      </label>
                      <select
                        value={{sortBy}}
                        onChange={{(e) => setSortBy(e.target.value)}}
                        className="w-full px-4 py-2 border border-slate-300 rounded-lg focus:ring-2 focus:ring-emerald-500 focus:border-transparent transition-all"
                      >
                        <option value="mentions">Most Mentioned</option>
                        <option value="price-asc">Price: Low to High</option>
                        <option value="price-desc">Price: High to Low</option>
                      </select>
                    </div>
                  </div>

                  <div className="mt-4 flex items-center justify-between">
                    <div className="text-sm text-slate-600">
                      Showing <span className="font-semibold text-slate-900">{{filteredStocks.length}}</span> stocks
                    </div>
                    {{(searchQuery || priceFilter !== 'all') && (
                      <button
                        onClick={{() => {{ setSearchQuery(''); setPriceFilter('all'); }}}}
                        className="text-sm text-emerald-600 hover:text-emerald-700 font-medium"
                      >
                        Clear Filters
                      </button>
                    )}}
                  </div>
                </div>

                {{/* Stock Grid */}}
                <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-4 md:gap-6">
                  {{filteredStocks.map((stock, idx) => (
                    <div
                      key={{stock.ticker}}
                      className="bg-white rounded-xl shadow-md hover:shadow-xl transition-all duration-300 overflow-hidden cursor-pointer transform hover:scale-105 fade-in"
                      style={{{{ animationDelay: `${{idx * 0.05}}s` }}}}
                      onClick={{() => setSelectedStock(stock)}}
                    >
                      <div className="p-6">
                        <div className="flex items-start justify-between mb-4">
                          <div className="flex-1">
                            <h3 className="text-2xl font-bold text-slate-900 mb-1">{{stock.ticker}}</h3>
                            <div className="flex items-center gap-2">
                              <DollarSign size={{18}} className="text-emerald-600" />
                              <span className="text-3xl font-bold text-emerald-600">
                                {{stock.price.toFixed(2)}}
                              </span>
                            </div>
                          </div>

                          <div className={{`px-3 py-1 rounded-full text-xs font-semibold ${{
                            stock.price < 10 ? 'bg-emerald-100 text-emerald-700' :
                            stock.price < 50 ? 'bg-blue-100 text-blue-700' :
                            'bg-purple-100 text-purple-700'
                          }}`}}>
                            {{stock.price < 10 ? 'Penny' : stock.price < 50 ? 'Growth' : 'Blue Chip'}}
                          </div>
                        </div>

                        <div className="flex items-center gap-2 mb-4 text-sm text-slate-600">
                          <FileText size={{16}} />
                          <span>
                            <strong>{{stock.source_count}}</strong> article{{stock.source_count !== 1 ? 's' : ''}}
                          </span>
                        </div>

                        <div className="bg-gradient-to-br from-slate-50 to-blue-50 rounded-lg p-4 mb-4">
                          <p className="text-sm text-slate-700 line-clamp-3">
                            {{stock.sources[0].notes}}
                          </p>
                        </div>

                        <button className="w-full px-4 py-2 bg-gradient-to-r from-emerald-600 to-emerald-700 hover:from-emerald-700 hover:to-emerald-800 text-white rounded-lg transition-all flex items-center justify-center gap-2 font-medium shadow-sm hover:shadow-md">
                          View Details
                          <ExternalLink size={{16}} />
                        </button>
                      </div>
                    </div>
                  ))}}
                </div>

                {{/* Empty State */}}
                {{filteredStocks.length === 0 && (
                  <div className="text-center py-16 fade-in">
                    <div className="text-6xl mb-4">📊</div>
                    <h3 className="text-xl font-semibold text-slate-900 mb-2">No stocks found</h3>
                    <p className="text-slate-600 mb-4">Try adjusting your filters</p>
                    <button
                      onClick={{() => {{ setSearchQuery(''); setPriceFilter('all'); }}}}
                      className="px-6 py-2 bg-emerald-600 hover:bg-emerald-700 text-white rounded-lg transition-colors"
                    >
                      Clear Filters
                    </button>
                  </div>
                )}}

                {{/* Modal */}}
                {{selectedStock && (
                  <div
                    className="fixed inset-0 bg-black bg-opacity-50 flex items-center justify-center p-4 z-50 fade-in"
                    onClick={{() => setSelectedStock(null)}}
                  >
                    <div
                      className="bg-white rounded-2xl max-w-4xl w-full max-h-[90vh] overflow-y-auto shadow-2xl"
                      onClick={{(e) => e.stopPropagation()}}
                    >
                      <div className="sticky top-0 bg-white border-b border-slate-200 p-6 md:p-8 z-10">
                        <div className="flex items-start justify-between">
                          <div className="flex-1">
                            <h2 className="text-3xl md:text-4xl font-bold text-slate-900 mb-2">{{selectedStock.ticker}}</h2>
                            <div className="flex items-center gap-3">
                              <span className="text-3xl md:text-4xl font-bold text-emerald-600">
                                ${{selectedStock.price.toFixed(2)}}
                              </span>
                              <div className={{`px-3 py-1 rounded-full text-sm font-semibold ${{
                                selectedStock.price < 10 ? 'bg-emerald-100 text-emerald-700' :
                                selectedStock.price < 50 ? 'bg-blue-100 text-blue-700' :
                                'bg-purple-100 text-purple-700'
                              }}`}}>
                                {{selectedStock.price < 10 ? 'Penny Stock' : selectedStock.price < 50 ? 'Growth Stock' : 'Blue Chip'}}
                              </div>
                            </div>
                          </div>
                          <button
                            onClick={{() => setSelectedStock(null)}}
                            className="text-slate-400 hover:text-slate-600 transition-colors p-2 hover:bg-slate-100 rounded-lg"
                          >
                            <X size={{24}} />
                          </button>
                        </div>
                      </div>

                      <div className="p-6 md:p-8">
                        <div className="space-y-6">
                          <div>
                            <h3 className="text-xl font-semibold text-slate-900 mb-4 flex items-center gap-2">
                              <FileText size={{24}} className="text-emerald-600" />
                              Featured in {{selectedStock.source_count}} Article{{selectedStock.source_count !== 1 ? 's' : ''}}
                            </h3>
                          </div>

                          {{selectedStock.sources.map((source, idx) => (
                            <div key={{idx}} className="bg-gradient-to-br from-slate-50 to-blue-50 rounded-xl p-6 hover:shadow-md transition-shadow">
                              <div className="flex items-start justify-between mb-3">
                                <h4 className="font-semibold text-slate-900 flex-1 mr-4 text-lg">
                                  {{source.title}}
                                </h4>
                                <a
                                  href={{source.url}}
                                  target="_blank"
                                  rel="noopener noreferrer"
                                  className="text-emerald-600 hover:text-emerald-700 transition-colors p-2 hover:bg-white rounded-lg"
                                  onClick={{(e) => e.stopPropagation()}}
                                >
                                  <ExternalLink size={{20}} />
                                </a>
                              </div>

                              <div className="text-sm text-slate-500 mb-4">
                                Published: {{source.published || 'N/A'}}
                              </div>

                              <div className="bg-white rounded-lg p-4 border-l-4 border-emerald-500">
                                <p className="text-slate-700 leading-relaxed">
                                  {{source.notes}}
                                </p>
                              </div>
                            </div>
                          ))}}
                        </div>

                        <button
                          onClick={{() => setSelectedStock(null)}}
                          className="mt-6 w-full px-6 py-3 bg-slate-200 hover:bg-slate-300 text-slate-900 rounded-lg transition-colors font-medium"
                        >
                          Close
                        </button>
                      </div>
                    </div>
                  </div>
                )}}
              </div>
            </div>
          );
        }}

        const root = ReactDOM.createRoot(document.getElementById('root'));
        root.render(<StockScreener />);
    </script>
</body>
</html>"""

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(html_content)

    abs_path = os.path.abspath(output_file)
    print(f"✅ Generated: {abs_path}")
    return abs_path

# Generate and launch
html_path = generate_interactive_stock_screener(ui_stocks, 'stock_screener_interactive.html')
webbrowser.open('file://' + html_path)

print(f"\n🚀 Interactive stock screener opened in browser!")
print(f"📁 File: {html_path}")
print(f"🎯 Features: Live filtering, sorting, search, stats dashboard, animations")

✅ Generated: /Users/nirmalmuppiri/Documents/stock_filtering_app/stock_screener_interactive.html

🚀 Interactive stock screener opened in browser!
📁 File: /Users/nirmalmuppiri/Documents/stock_filtering_app/stock_screener_interactive.html
🎯 Features: Live filtering, sorting, search, stats dashboard, animations
